# Calculate kW usage of solar panels per station
This notebook converts the projected kW usage per buurt to their associated station by taking the nearest Euclidean distance.

In [104]:
import pandas as pd
import numpy as np

stations = pd.read_csv("../data/final_stations.csv")
pc6 = pd.read_csv("../data/pc6hnr20170801_gwb.csv", sep=';')

solar_panels = {}
for year in np.arange(2016, 2030):
    solar_panels[year] = pd.read_csv(f'../data/predictions/predictions_{year}.csv')
    #solar_panels[year] = solar_panels[year][(solar_panels[year]['WijkenEnBuurten'] != 141303) & (solar_panels[year]['WijkenEnBuurten'] != 140006)]

We convert buurt codes to postcode and then convert postcode to latlong and then latlong to nearest station ID.
First, we define some functions to convert postcodes to longlats and a function to loop over postcodes in a pd Group.

In [105]:
postcode_url = 'https://basisregistraties.arcgisonline.nl/arcgis/rest/services/BAG/BAGv2/MapServer/0/query?where=postcode%3D%27{postcode}%27&text=&objectIds=&time=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=4326&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=1&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=pjson'

import requests

def postcode_to_longlat(postcode):
    result = requests.get(postcode_url.format(postcode=postcode)).json()
    if len(result['features']) < 1:
        raise ValueError(f'postcode {postcode} yielded no results')
    return list(result['features'][0]['geometry'].values())[::-1]

def postcode_group_to_longlat(postcodes):
    for index, row in postcodes.iterrows():
        try:
            latlong = postcode_to_longlat(row.PC6)
        except:
            continue
            
        return latlong
    print(postcodes)
    raise Exception("Bestaat nie")

Here, we loop over every year and calculate the nearest station to add the kW to. We save these as json objects to disk.

In [106]:
from collections import defaultdict
import json

for year, solar_ps in solar_panels.items():
    panels_per_pc6 = solar_ps.merge(pc6, how='left', left_on='WijkenEnBuurten', right_on='Buurt2017')
    latlongs = panels_per_pc6.groupby('WijkenEnBuurten').apply(postcode_group_to_longlat)

    panels_per_pc6 = panels_per_pc6.merge(pd.DataFrame(latlongs, columns=['latlong']), how='left', on='WijkenEnBuurten').groupby('WijkenEnBuurten').first()
    
    station_pos = stations[['LAT', 'LONG']].values

    to_add = defaultdict(int)

    for index, row in panels_per_pc6.iterrows():
        closest_station = np.argmin(np.linalg.norm(row['latlong'] - station_pos, axis=1))

        to_add[str(closest_station)] += row['predicted_nr_panels'] * 0.1 # watt
    
    with open(f'toadd_{year}.json', 'w') as f:
        json.dump(to_add, f)